In [1]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise
from fair.earth_params import seconds_per_year # Don't really neeed this

## Initialise FaIR and define time horizon
1. Makes a FaIR object and the methane life time method. Version 2 uses the version from Nick Leaches paper.
   By default this is the one that is used but you can specify it if you put 'leach2021' in here.
2. By using 'thornhill2021' we are using the version with interactive methane lifetime. 

In [2]:
import fair
fair.__version__

'2.2.0'

In [3]:
f = FAIR()

In [4]:
f.ch4_method='thornhill2021'

In [5]:
help(f.define_time)

Help on method define_time in module fair.fair:

define_time(start, end, step) method of fair.fair.FAIR instance
    Define timebounds vector to run FaIR.
    
    Parameters
    ----------
    start : float
        first timebound of the model (year)
    end : float
        last timebound of the model (year)
    step : float
        timestep (year)



In [6]:
f.define_time(1750, 2500, 1)

## 2. Define scenarios
   Focusing on 4 of the SSPs; important that the names are consistent with those in the RCMIP database

In [7]:
# Define SSP scenarios
#scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp534-over', 'ssp585']
scenarios = [ 'ssp126', 'ssp245', 'ssp370', 'ssp585']
f.define_scenarios(scenarios)
print (f.scenarios)

['ssp126', 'ssp245', 'ssp370', 'ssp585']


## 3. Define configs 
   Here we are using the AR6 defined ones specified in 1.4.0

In [8]:
df_configs = pd.read_csv('../data/fair-parameters/calibration-AR6/calibrated_constrained_parameters_1.4.0.csv', index_col=0)

In [9]:
f.define_configs(df_configs.index)

## 4. Define species and properties 

In [10]:
species, properties = read_properties('../data/fair-parameters/calibration-AR6/species_configs_properties_1.4.0.csv')
f.define_species(species, properties)

## 5. Create input and output data

In [11]:
f.allocate()

## 6. Fill in the data

    we read in a default list of species configs that will apply to each config. If you want to change specific configs then you can still use this function to set defaults and tweak what you need. We will do this with the methane lifetime, which has a different value calibrated for the Thornhill 2021 lifetime option.

Also going to subtract the RCMIP 1750 emissions from CH4 and N2O. This is not in the default configs.

In [12]:
f.fill_species_configs()
fill(f.species_configs['unperturbed_lifetime'], 10.8537568, specie='CH4')
fill(f.species_configs['baseline_emissions'], 19.01978312, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

## 6a. Now get emissions

 emissions (+solar and volcanic forcing) from RCMIP datasets using the fill_from_rcmip helper function. This function automatically selects the emissions, concentration or forcing you want depending on the properties for each of the SSP scenarios defined.

Replace the volcanic dataset with the AR6 volcanic dataset, as I want to compare the impact of monthly volcanic forcing in the monthly comparison.


You can modify the input_mode to be concentrations if you don't have all the gases as emissions. 
You have to have both fossil emissions and AFOLU emissions to run in emissions mode. 

The following creates xarrays that you need. so running f.emissions now will return the empty arrays prepared for running FaIR. 

In [13]:
volcanic_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-calibrate/main/data/forcing/volcanic_ERF_1750-2101_timebounds.csv',
    known_hash = 'md5:c0801f80f70195eb9567dbd70359219d',
)

In [14]:
solar_obj = pooch.retrieve(
    url = 'https://raw.githubusercontent.com/chrisroadmap/fair-add-hfc/main/data/solar_erf_timebounds.csv',
    known_hash = 'md5:98f6f4c5309d848fea89803683441acf',
)

In [15]:
df_solar = pd.read_csv(solar_obj, index_col="year")
df_volcanic = pd.read_csv(volcanic_obj)

In [16]:
print(df_volcanic)
print(df_solar)

     timebounds       erf
0          1750  0.246391
1          1751  0.246412
2          1752  0.246420
3          1753  0.246422
4          1754  0.246423
..          ...       ...
347        2097  0.000000
348        2098  0.000000
349        2099  0.000000
350        2100  0.000000
351        2101  0.000000

[352 rows x 2 columns]
           erf
year          
1750  0.102215
1751  0.097800
1752  0.079454
1753  0.049376
1754  0.013198
...        ...
2296 -0.004042
2297  0.019260
2298  0.044293
2299  0.044941
2300  0.025859

[551 rows x 1 columns]


In [17]:
f.emissions

<xarray.DataArray (timepoints: 750, scenario: 4, config: 841, specie: 61)> Size: 1GB
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]], shape=(750, 4, 841, 61))
Coordinates:
  * timepoints  (timepoints) float64 6kB 1.75e+03 1.752e+03 ... 2.5e+03
  * scenario    (scenario) <U6 96B 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * config      (config) int64 7kB 2463 2658 4204 ... 1595003 1595963 1597740
  * specie      (specie) <U43 10kB 'CO2 FFI' ... 'Equivalent effective strato...

Read_properties is imported at the beginning, takes list of default species and properties including GHGs, shortlived forcing and anything you want to calculate and anything you want to calculate a forcing for (so any of the 40 species and things like aerosol cloud forcing).

Read in RCMIP emissions from zenodo stores it in cache and populates the emissions with rcmip data. 

In [18]:
f.fill_from_rcmip()

In [19]:
f.configs

Index([   2463,    2658,    4204,    4743,    6941,    7267,    9765,   10917,
         13465,   14044,
       ...
       1583204, 1584567, 1584895, 1588527, 1589426, 1592375, 1593438, 1595003,
       1595963, 1597740],
      dtype='int64', length=841)

Running f.emissions should not be empty anymore

In [20]:
f.emissions

<xarray.DataArray (timepoints: 750, scenario: 4, config: 841, specie: 61)> Size: 1GB
array([[[[0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan]],

        [[0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
         [0.00950562, 0.29746461,        nan, ...,        nan,
                 nan,        nan],
...
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan]],

        [[0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan],
         [0.        , 0.        ,        nan, ...,        nan,
                 nan,        nan]]]], shape=(750, 4, 841, 61))
Coordinates:
  * timepoints  (timepoints) float64 6kB 1.75e+03 1.752e+03 ... 2.5e+03
  * scenario    (scenario) <U6 96B 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * config      (config) int64 7kB 2463 2658 4204 ... 1595003 1595963 1597740
  * specie      (specie) <U43 10kB 'CO2 FFI' ... 'Equivalent effective strato...

We also need to initialise the first timestep of the run in terms of its per-species forcing, temperature, cumulative and airborne emissions. We set these all to zero. The concentration in the first timestep will be set to the baseline concentration, which are the IPCC AR6 1750 values.

In [21]:
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

We can define natural variability using stochastic_run if we want! We do this by using the code in the box below this next one.

Print out the species used here:

In [22]:
f.species

['CO2 FFI',
 'CO2 AFOLU',
 'CO2',
 'CH4',
 'N2O',
 'Sulfur',
 'BC',
 'OC',
 'NH3',
 'NOx',
 'VOC',
 'CO',
 'CFC-11',
 'CFC-12',
 'CFC-113',
 'CFC-114',
 'CFC-115',
 'HCFC-22',
 'HCFC-141b',
 'HCFC-142b',
 'CCl4',
 'CHCl3',
 'CH2Cl2',
 'CH3Cl',
 'CH3CCl3',
 'CH3Br',
 'Halon-1211',
 'Halon-1301',
 'Halon-2402',
 'CF4',
 'C2F6',
 'C3F8',
 'c-C4F8',
 'C4F10',
 'C5F12',
 'C6F14',
 'C7F16',
 'C8F18',
 'NF3',
 'SF6',
 'SO2F2',
 'HFC-125',
 'HFC-134a',
 'HFC-143a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-23',
 'HFC-236fa',
 'HFC-245fa',
 'HFC-32',
 'HFC-365mfc',
 'HFC-4310mee',
 'Solar',
 'Volcanic',
 'Aerosol-radiation interactions',
 'Aerosol-cloud interactions',
 'Ozone',
 'Light absorbing particles on snow and ice',
 'Stratospheric water vapour',
 'Land use',
 'Equivalent effective stratospheric chlorine']

## 7c. fill climate configs¶

In the example Chris uses pre-calculated values from the Cummins et al. three layer model and use a reproducible random seed to define the stochastic behaviour as below.

        df = pd.read_csv("../tests/test_data/4xCO2_cummins_ebm3.csv")
        models = df['model'].unique()
        
        seed = 1355763
        
        for config in configs:
            model, run = config.split('_')
            condition = (df['model']==model) & (df['run']==run)
            fill(f.climate_configs['ocean_heat_capacity'], df.loc[condition, 'C1':'C3'].values.squeeze(), config=config)
            fill(f.climate_configs['ocean_heat_transfer'], df.loc[condition, 'kappa1':'kappa3'].values.squeeze(), config=config)
            fill(f.climate_configs['deep_ocean_efficacy'], df.loc[condition, 'epsilon'].values[0], config=config)
            fill(f.climate_configs['gamma_autocorrelation'], df.loc[condition, 'gamma'].values[0], config=config)
            fill(f.climate_configs['sigma_eta'], df.loc[condition, 'sigma_eta'].values[0], config=config)
            fill(f.climate_configs['sigma_xi'], df.loc[condition, 'sigma_xi'].values[0], config=config)
            fill(f.climate_configs['stochastic_run'], True, config=config)
            fill(f.climate_configs['use_seed'], True, config=config)
            fill(f.climate_configs['seed'], seed, config=config)
            
            seed = seed + 399

However I thought that the configs should be in the 1.4.0 files which are in ../data/fair-parameters/calibration-AR6/calibrated_constrained_parameters_1.4.0.csv

 Printing out the configs using df_configs looks like it has the right stuff in it.

In [23]:
df_configs 

,gamma_autocorrelation,ocean_heat_capacity[0],ocean_heat_capacity[1],ocean_heat_capacity[2],ocean_heat_transfer[0],ocean_heat_transfer[1],ocean_heat_transfer[2],deep_ocean_efficacy,sigma_eta,sigma_xi,...,forcing_scale[Stratospheric water vapour],forcing_scale[Land use],forcing_scale[Volcanic],forcing_scale[Solar],forcing_scale[Light absorbing particles on snow and ice],forcing_scale[CO2],baseline_concentration[CO2],seed,stochastic_run,use_seed
2463,1.659270,4.251980,23.326194,65.743426,1.017097,1.034456,0.965522,0.828150,0.551572,0.438131,...,1.256213,0.321879,1.058597,1.072253,0.558304,1.006500,278.448530,2338500,False,False
2658,10.468831,3.762726,15.584639,174.733259,1.800834,2.879661,0.587163,0.851757,1.453361,0.770893,...,0.383282,0.914854,0.744196,0.458884,1.581370,1.111147,276.858025,2416305,False,False
4204,2.164340,6.021519,17.738140,53.532316,1.324483,2.069387,1.907546,1.933912,0.730088,0.508479,...,0.726873,0.495631,0.774407,0.901991,0.838553,1.001782,278.096333,3033159,False,False
4743,9.803904,4.203418,23.590344,39.443375,0.772692,1.844115,1.434551,1.141965,0.846429,0.453668,...,3.213142,0.758653,0.745282,0.783342,2.093639,0.925876,275.912349,3248220,False,False
6941,6.938420,3.689112,28.758447,70.481777,1.085572,1.956576,0.717242,1.103431,0.632468,0.453343,...,1.435408,0.623485,1.127848,1.155475,1.836414,0.935305,276.713510,4125222,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592375,4.230229,3.645421,5.642910,85.490494,1.477319,2.606770,1.593995,1.227703,0.925179,0.389785,...,0.103811,0.886405,0.860350,1.154834,1.708537,0.929476,280.719974,636713388,False,False
1593438,4.403917,6.547364,21.183257,173.620098,1.319879,2.458949,0.902300,1.134010,0.633675,0.666582,...,2.234205,0.838154,0.932193,0.472826,0.893812,0.992307,277.755534,637137525,False,False
1595003,2.878047,5.743957,14.708426,36.690305,1.159164,2.983215,1.653740,1.842435,0.624423,0.474596,...,0.780387,0.383184,1.199521,1.240175,0.560291,1.001767,277.331130,637761960,False,False
1595963,5.734443,4.130985,15.721242,91.737866,1.335746,2.630239,1.157261,1.306096,1.362210,0.484448,...,0.651498,0.784874,1.007218,1.429140,0.661469,0.909005,277.460189,638145000,False,False


In [24]:
f.run()

ValueError: There are NaN values in FAIR.climate_configs['ocean_heat_capacity']

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ssp245', layer=0)], label=f.configs);
pl.title('ssp245: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')

In [ ]:
dir(f)

Defining time in emissions you need to remember to go from the midpoint of the year you are interested in, so go 2015.5 to get 2015.

In [ ]:
f.emissions.loc[
    dict(
        scenario='ssp245', 
        specie='CH4', 
        timepoints=np.arange(2015.5, 2500.5)
    )
]

In [ ]:
f.emissions.loc[
    dict(
        scenario='ssp245', 
        specie='CH4', 
        timepoints=np.arange(2015.5, 2500.5)
    )
]=0.2*f.emissions.loc[
    dict(
        scenario='ssp245', 
        specie='CH4', 
        timepoints=np.arange(2015.5, 2500.5)
    )
]

In [ ]:
f.emissions.loc[
    dict(
        scenario='ssp245', 
        specie='CH4', 
        timepoints=np.arange(2015.5, 2500.5)
    )
]

In [ ]:
print(f.ocean_heat_content_change.loc[dict(scenario='ssp245', timebounds=np.arange(1750, 2500))])

In [ ]:
print(f.temperature.loc[dict(scenario='ssp245', timebounds=np.arange(1750, 2500))])

In [ ]:
# Output to csv, first create a dataframe 
model_archive = '../csv_FaIRoutputfiles_ssp/'
print (scenarios)
print 

if os.path.isdir(model_archive):
    print("Item already exists")
else:
    os.mkdir(model_archive)
    print ("Making the model directory")
    
#model_dir = model_archive+scenario+'/'
    
for scenario in scenarios:
    for variable in ['ocean_heat_content_change', 'temperature']:
    
        years = 1750+np.arange(len(f.temperature.loc[dict(scenario=scenario, timebounds=np.arange(1750, 2501))]))
        header_text = np.insert(years, 0, 0)
        if variable == 'ocean_heat_content_change':
            dic = f.ocean_heat_content_change.loc[dict(scenario=scenario, timebounds=np.arange(1750, 2501))]
        else:
            dic = f.temperature.loc[dict(scenario=scenario, timebounds=np.arange(1750, 2501), layer=0)]
        
        SSPs = {'ssp119':'ssp119',
                'ssp126':'ssp126',
                'ssp245':'ssp245',
                'ssp370':'ssp370',
                'ssp534-over':'ssp534_over', 
                'ssp585':'ssp585'
          }
        
        filename='../csv_FaIRoutputfiles_ssp/ssprcmip_'+SSPs[scenario]+'_'+variable+'.csv'
        print(filename)
        pd.DataFrame(dic).T.reset_index().to_csv(filename, header=header_text, index=False)
        

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ssp126', layer=0)], label=f.configs);
pl.title('ssp126: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')

In [ ]:
pl.plot(f.timebounds, f.temperature.loc[dict(scenario='ssp585', layer=0)], label=f.configs);
pl.title('ssp585: temperature')
pl.xlabel('year')
pl.ylabel('Temperature anomaly (K)')

In [ ]:
pl.plot(f.timebounds, f.ocean_heat_content_change.loc[dict(scenario='ssp585')], label=f.configs);
pl.title('ssp585: Ocean heat content change')
pl.xlabel('year')
pl.ylabel('J')


In [ ]:
pl.plot(f.timebounds, f.ocean_heat_content_change.loc[dict(scenario='ssp126')], label=f.configs);
pl.title('ssp126: Ocean heat content change')
pl.xlabel('year')
pl.ylabel('J')

In [ ]:
for perc in [5,25,50,75,100]:
    pl.plot(f.timebounds[:352], np.percentile(f.temperature.loc[dict(scenario='ssp585', layer=0)],perc, axis=1)[:352], label=f.configs)
    pl.title('ssp585: temperature')
    pl.xlabel('year')
    pl.ylabel('Temperature anomaly (K)')
    

In [ ]:
for perc in [0, 5,25,50,75,100]:
    pl.plot(f.timebounds[:352], np.percentile(f.temperature.loc[dict(scenario='ssp245', layer=0)],perc, axis=1)[:352], label=f.configs)
    pl.title('ssp245: temperature')
    pl.xlabel('year')
    pl.ylabel('Temperature anomaly (K)')

In [ ]:
for perc in [0, 5,25,50,75,100]:
    pl.plot(f.timebounds[:352], np.percentile(f.temperature.loc[dict(scenario='ssp370', layer=0)],perc, axis=1)[:352], label=f.configs)
    pl.title('ssp370: temperature')
    pl.xlabel('year')
    pl.ylabel('Temperature anomaly (K)')

In [ ]:
print(f.timebounds[270],f.temperature.loc[dict(scenario='ssp370', layer=0)][270])